In [38]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from PIL import Image
from pathlib import Path
from matplotlib.patches import Rectangle
from scipy.io import loadmat

# Stanford Cars dataset reading

In [43]:
devkit_path = Path('StanfordCarsDataset/devkit')
train_path = Path('StanfordCarsDataset/cars_train')
test_path = Path('StanfordCarsDataset/cars_test')

## DevKit

In [40]:
os.listdir(devkit_path)

['cars_meta.mat',
 'cars_test_annos.mat',
 'cars_train_annos.mat',
 'eval_train.m',
 'README.txt',
 'train_perfect_preds.txt']

In [42]:
with open(devkit_path/'README.txt') as f:
    for line in f:
        print(line)

This file gives documentation for the cars 196 dataset.

(http://ai.stanford.edu/~jkrause/cars/car_dataset.html)



----------------------------------------

Metadata/Annotations

----------------------------------------

Descriptions of the files are as follows:



-cars_meta.mat:

  Contains a cell array of class names, one for each class.



-cars_train_annos.mat:

  Contains the variable 'annotations', which is a struct array of length

  num_images and where each element has the fields:

    bbox_x1: Min x-value of the bounding box, in pixels

    bbox_x2: Max x-value of the bounding box, in pixels

    bbox_y1: Min y-value of the bounding box, in pixels

    bbox_y2: Max y-value of the bounding box, in pixels

    class: Integral id of the class the image belongs to.

    fname: Filename of the image within the folder of images.



-cars_test_annos.mat:

  Same format as 'cars_train_annos.mat', except the class is not provided.



----------------------------------------

Submiss

In [ ]:
cars_meta = loadmat(devkit_path/'cars_meta.mat')
cars_train_annos = loadmat(devkit_path/'cars_train_annos.mat')